#Grupo 25 - Entregable Aprendizaje Supervisado

#Parte 2 de 3

Integrantes:

Rufeil Fiori, Elena

Sosa, Manuel

Taié Mirón, Macarena

Yurovich, Luis Federico

# Preprocesamiento

Resumen:

- Ingenieria de características: Separación Cabin, agregado de cabin_qty y total_ammenities
- Filtrado de valores extremos: no efectuamos
- Codificación de valores categóricos
- Escalado 
- Imputación de valores nulos



Importamos las librerías necesarias para el prepocesamiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url_test = 'https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/test.csv'
url_train = 'https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/train.csv'
df_train = pd.read_csv(url_train, sep = ',')
#df_test = pd.read_csv(url_test, sep = ',')

Generamos una copia del data frame del conjunto de entrenamiento, al que le aplicaremos el preprocesamiento. Posteriormente, realizamos los mismos pasos al conjunto de test, cambiando la asignación del "df" inicial.

In [ ]:
df = df_train.copy()
#df = df_test.copy()
print(df.shape)
df.head()

(7823, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,3430,Europa,False,C/123/P,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,Azhah Tronsive,0
1,669,Mars,True,F/126/S,NaN,18.0,False,0.0,0.0,0.0,0.0,0.0,Roswal Sha,1
2,8430,Europa,True,B/343/S,TRAPPIST-1e,33.0,NaN,0.0,0.0,0.0,0.0,0.0,Shaiam Tinganking,1
3,6463,Earth,False,F/1420/P,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,Landa Grifford,0
4,85,Earth,False,G/9/P,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,Gracy Dunnisey,0


Convertimos en index a la variable PassengerId para no involucrarla como input en los modelos de machine learning, pero al mismo tiempo conservar el vínculo con sus atributos.

In [ ]:
df = df.set_index("PassengerId")
df.sample(10)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
3074,Europa,False,B/123/S,TRAPPIST-1e,39.0,False,0.0,957.0,0.0,22.0,401.0,Acrabih Openduave,1
3639,Earth,False,F/738/S,NaN,23.0,False,0.0,93.0,1.0,0.0,542.0,Catha Waterson,0
7185,Earth,True,G/1247/S,PSO J318.5-22,49.0,False,0.0,0.0,0.0,0.0,0.0,Alber Puckenton,0
1645,Europa,False,E/130/S,TRAPPIST-1e,28.0,False,0.0,1310.0,0.0,221.0,6064.0,NaN,0
214,Mars,False,F/42/S,TRAPPIST-1e,32.0,True,181.0,0.0,5.0,1634.0,0.0,Blues Queen,0
607,Earth,False,F/136/P,55 Cancri e,13.0,False,0.0,7.0,0.0,339.0,359.0,Craine Santantines,0
273,Earth,False,F/67/P,TRAPPIST-1e,23.0,False,0.0,0.0,15.0,0.0,1370.0,Lesley Shephendry,0
6240,Earth,False,F/1265/S,TRAPPIST-1e,62.0,False,94.0,267.0,2341.0,94.0,0.0,Davis Cookson,1
6208,Earth,False,G/1067/S,TRAPPIST-1e,30.0,False,0.0,123.0,0.0,1.0,784.0,Juanna Gainney,0


## Ingeniería de características

A partir de la variable "Cabin" agregamos tres atributos: "Deck", "Num" y "Side" que corresponden a la cubierta, número de cabina y lado de la nave (babor y estribor). Además, generamos una columna que corresponde a la cantidad de personas que se encuentran en dicha cabina: "Cabin_qty".

In [ ]:
df = df.merge(df.Cabin.str.split('/', expand = True).rename(columns = dict(zip([0,1,2],["Deck", "Num", "Side"]))), 
         left_index=True, right_index=True)
df["Num"] = df["Num"].astype(float)

In [ ]:
cabin_groups = df.Cabin.value_counts()
df["Cabin_qty"] = df[df.Cabin.notnull()].Cabin.apply(lambda c: cabin_groups.loc[c])

Por otro lado, generamos una nueva variable, "Total Ammenities", a partír de la suma de las variables de los servicios de lujo: "RoomService", "FoodCourt", "ShoppingMall", "Spa" y "VRDeck".

In [ ]:
ammenities_columns = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
df["Total_ammenities"] = df[ammenities_columns].sum(axis = 1)

In [ ]:
df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Cabin_qty,Total_ammenities
PassengerId,,,,,,,,,,,,,,,,,,
3430,Europa,False,C/123/P,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,Azhah Tronsive,0,C,123.0,P,4.0,7941.0
669,Mars,True,F/126/S,NaN,18.0,False,0.0,0.0,0.0,0.0,0.0,Roswal Sha,1,F,126.0,S,1.0,0.0
8430,Europa,True,B/343/S,TRAPPIST-1e,33.0,NaN,0.0,0.0,0.0,0.0,0.0,Shaiam Tinganking,1,B,343.0,S,1.0,0.0
6463,Earth,False,F/1420/P,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,Landa Grifford,0,F,1420.0,P,1.0,1554.0
85,Earth,False,G/9/P,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,Gracy Dunnisey,0,G,9.0,P,3.0,705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528,Mars,False,F/1319/S,TRAPPIST-1e,12.0,NaN,0.0,0.0,0.0,0.0,0.0,Risps Hacle,1,F,1319.0,S,3.0,0.0
2693,Earth,False,G/459/S,TRAPPIST-1e,44.0,False,0.0,0.0,744.0,62.0,3.0,Gabrie Reenez,0,G,459.0,S,1.0,809.0
8076,Mars,True,F/1772/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Panzel Ruity,1,F,1772.0,P,1.0,0.0


## Filtrado de valores extremos

Decidimos no filtrar outliers ya que consideramos que estos datos caracterizan a la población, brindando información para nuestros modelos.

## Codificación de valores categóricos

Para la variable "Deck" aplicamos una transformación ordinal que convierte de {A, B, C, D, E, F, G, T} a {0, 1, 2, 3, 4, 5, 6, 7}, respectivamente.

Las variables binarias "VIP" con valores {False, True}, "Side" con {S, P} y "CryoSleep" con {False, True}, son transformadas a {0, 1}, respectivamente.

Las variables "HomePlanet" y "Destination" fueron transformadas utilizando el método One Hot Encoding. En estos casos, hemos optado por dejar los valores nulos de estas variables como una nueva categoría, ya que creemos que la imputación en estas variables podría generar ruido.

In [ ]:
df_processed = df.copy()
#df_processed.index.name = "PassengerId"
df_processed

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Cabin_qty,Total_ammenities
PassengerId,,,,,,,,,,,,,,,,,,
3430,Europa,False,C/123/P,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,Azhah Tronsive,0,C,123.0,P,4.0,7941.0
669,Mars,True,F/126/S,NaN,18.0,False,0.0,0.0,0.0,0.0,0.0,Roswal Sha,1,F,126.0,S,1.0,0.0
8430,Europa,True,B/343/S,TRAPPIST-1e,33.0,NaN,0.0,0.0,0.0,0.0,0.0,Shaiam Tinganking,1,B,343.0,S,1.0,0.0
6463,Earth,False,F/1420/P,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,Landa Grifford,0,F,1420.0,P,1.0,1554.0
85,Earth,False,G/9/P,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,Gracy Dunnisey,0,G,9.0,P,3.0,705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528,Mars,False,F/1319/S,TRAPPIST-1e,12.0,NaN,0.0,0.0,0.0,0.0,0.0,Risps Hacle,1,F,1319.0,S,3.0,0.0
2693,Earth,False,G/459/S,TRAPPIST-1e,44.0,False,0.0,0.0,744.0,62.0,3.0,Gabrie Reenez,0,G,459.0,S,1.0,809.0
8076,Mars,True,F/1772/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Panzel Ruity,1,F,1772.0,P,1.0,0.0


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [ ]:
deck_ordinal = enc.fit_transform(df["Deck"].values.reshape(-1,1))
df_processed["Deck"] = pd.Series(deck_ordinal.flatten())

In [ ]:
side_encoded = enc.fit_transform(df_processed["Side"].values.reshape(-1,1))
df_processed["Side"] = pd.Series(side_encoded.flatten())

In [ ]:
for c in ["VIP", "CryoSleep"]:
  df_processed[c] = df_processed[c].astype(float)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder()

In [ ]:
df_processed.shape

(7823, 18)

HomePlanet

imputamos esta variable con la MODA

In [ ]:
#mode_homeplanet = df_processed["HomePlanet"].mode()
#df_processed["HomePlanet"].fillna(mode_homeplanet[0], inplace = True)

In [ ]:
homeplanet_enc = one_hot_enc.fit_transform(df_processed["HomePlanet"].values.reshape(-1,1))


In [ ]:
homeplanet_enc.toarray().shape

(7823, 4)

In [ ]:
df_processed.loc[:,[f"HomePlanet_{cat}" for cat in one_hot_enc.categories_[0]]] = homeplanet_enc.toarray()

In [ ]:
df_processed.shape

(7823, 22)

Destination

Imputamos nulos con la moda de Destination

In [ ]:
#mode_destination = df_processed["Destination"].mode()
#df_processed["Destination"].fillna(mode_destination[0], inplace = True)

In [ ]:
destination_enc = one_hot_enc.fit_transform(df_processed["Destination"].values.reshape(-1,1))
df_processed.loc[:, [f"Destination_{cat}" for cat in one_hot_enc.categories_[0]]] = destination_enc.toarray()

In [ ]:
df_processed.shape

(7823, 26)

Continuando con la ingeniería de características, eliminamos del data frame las variables "HomePlanet", "Destination" y "Cabin", ya que al tener su codificación correspondiente, no es necesario conservarlas. Asimismo, eliminamos la variable "Name" ya que consideramos que su información no es relevante para la predicción.

In [ ]:
df_processed = df_processed.drop(columns = ["HomePlanet", "Destination", "Cabin", "Name"])

In [ ]:
df_processed.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Transported', 'Deck', 'Num', 'Side', 'Cabin_qty',
       'Total_ammenities', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'HomePlanet_nan', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e',
       'Destination_nan'],
      dtype='object')

## Escalado de los datos

Elegimos escalar los datos usando el método MinMaxScaler, debido a que con este método obtuvimos los mejores valores de accuracy.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_processed), columns = df_processed.columns,
                         index = df_processed.index)

In [ ]:
df2=df_scaled


In [ ]:
df2

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,...,Cabin_qty,Total_ammenities,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
3430,0.0,0.721519,0.0,0.000000,0.129440,0.000553,0.008702,0.160527,0.0,0.571429,...,0.428571,0.220663,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
669,1.0,0.227848,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
8430,1.0,0.417722,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,NaN,...,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6463,0.0,0.544304,0.0,0.009911,0.040519,0.000000,0.000000,0.008453,0.0,0.714286,...,0.000000,0.043182,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
85,0.0,0.240506,0.0,0.000000,0.000000,0.028520,0.000045,0.001409,0.0,0.714286,...,0.285714,0.019590,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528,0.0,0.151899,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.285714,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2693,0.0,0.556962,0.0,0.000000,0.000000,0.031670,0.002767,0.000124,0.0,0.714286,...,0.000000,0.022480,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8076,1.0,0.696203,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,NaN,...,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


##  Imputación de valores nulos


Las siguientes variables son imputadas por su valor más frecuente:

- CryoSleep
- VIP


Las siguientes variables son imputadas por su mediana:
- Age
- RoomService
- FoodCourt
- ShoppingMall
- Spa
- VRDeck
- Cabin_qty


Mientras que para las siguientes variables, utilizamos el método KNN Imputer:

- Deck
- Num
- Side

In [ ]:
from sklearn.impute import SimpleImputer

simple = SimpleImputer(strategy='most_frequent')

imputar_mas_frecuente = ['CryoSleep','VIP'] 

mode = simple.fit_transform(df2.loc[:, imputar_mas_frecuente])


In [ ]:
imputar_mediana=['Age','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', "Cabin_qty"]

imputer= SimpleImputer(strategy="median")

median= imputer.fit_transform(df2.loc[:, imputar_mediana])


In [ ]:
col_knn_imputer = ["Deck", "Num", "Side"]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")

In [ ]:
df2 = pd.DataFrame(knn_imputer.fit_transform(df2),
             columns = df2.columns,
             index = df2.index)

In [ ]:
df2

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,...,Cabin_qty,Total_ammenities,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
3430,0.0,0.721519,0.0,0.000000,0.129440,0.000553,0.008702,0.160527,0.0,0.571429,...,0.428571,0.220663,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
669,1.0,0.227848,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
8430,1.0,0.417722,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.571429,...,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6463,0.0,0.544304,0.0,0.009911,0.040519,0.000000,0.000000,0.008453,0.0,0.714286,...,0.000000,0.043182,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
85,0.0,0.240506,0.0,0.000000,0.000000,0.028520,0.000045,0.001409,0.0,0.714286,...,0.285714,0.019590,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528,0.0,0.151899,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.285714,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2693,0.0,0.556962,0.0,0.000000,0.000000,0.031670,0.002767,0.000124,0.0,0.714286,...,0.000000,0.022480,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8076,1.0,0.696203,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.371429,...,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


# Guardamos los data frames preprocesados

In [ ]:
#df2.to_csv("/content/drive/MyDrive/DIPLODATOS 2022 grupo 25/Entregables/4 - Aprendizaje Supervisado/df test procesado.csv", index = True)

In [ ]:
#df2.to_csv("/content/drive/MyDrive/DIPLODATOS 2022 grupo 25/Entregables/4 - Aprendizaje Supervisado/df train procesado.csv", index = True)